In [1]:
from bs4 import BeautifulSoup
import requests
import re
import json
import pandas as pd

In [2]:
data = "20-08-2021"
url = 'https://www.in.gov.br/leiturajornal?data={}&secao=do3'.format(data)
r = requests.get(url, verify=False)
r.status_code

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


200

In [3]:
soup = BeautifulSoup(r.text, 'lxml')
mat = soup.find_all('script', type='application/json')[1].text
array = json.loads(mat)['jsonArray']
array  # olhando como estão os dados para definir o regex a seguir

[{'artType': 'Aviso de Prorrogação',
  'content': 'AVISO DE PRORROGAÇÃO PRORROGAÇÃO CONSULTA PÚBLICA SEAE Nº 2/2021 O Secretário de Advocacia da Concorrência e Competitividade, no uso das atribuições conferidas pelo art. 119, incisos I a IV, e art. 121, inciso III, do Decreto nº 9.745, de 8 de abril de 2019, COMUNICA que fica prorrogada por mais 30 (trinta) dias a data de encerramento da Consulta Pública SEAE nº 2/2021, publicada no Diário Oficial...',
  'hierarchyLevelSize': 3,
  'hierarchyList': ['Ministério da Economia',
   'Secretaria Especial de Produtividade, Emprego e Competitividade',
   'Secretaria de Advocacia da Concorrência e Competitividade'],
  'hierarchyStr': 'Ministério da Economia/Secretaria Especial de Produtividade, Emprego e Competitividade/Secretaria de Advocacia da Concorrência e Competitividade',
  'numberPage': '29',
  'pubOrder': 'DO300015:00050:00006:00000:00000:00000:00000:00000:00000:00000:00086:00000',
  'title': 'AVISO DE PRORROGAÇÃO',
  'urlTitle': 'aviso

In [4]:
contratante = pd.Series(re.findall(r'Contratante: ([\w+\sÀ-Ú]+)+', mat))
contratado = pd.Series(re.findall('Contratado: \d{2}\.\d{3}\.\d{3}/\d{4}\-\d{2} - ?([\w+\sÀ-Ú]+)+', mat))
cnpj = pd.Series(re.findall(r'CNPJ: (\d{2}\.\d{3}\.\d{3}/\d{4}\-\d{2})', mat))

contratos = pd.concat([contratante, contratado, cnpj], axis=1)
contratos.columns=["Contratante", "Contratado", "CNPJ"]
contratos.head(5)

,Contratante,Contratado,CNPJ
0,MUNICÍPIO DE PONTES E LACERDA Contratada,CNH INDUSTRIAL BRASIL LTDA,14.941.072/0001-48
1,PREFEITURA MUNICIPAL DE NOVA CANAÃ DO NORTE,AP PISCINAS EIRELI,15.180.714/0001-04
2,MUNICÍPIO DE PALOTINA,SINALES SINALIZAÇÃO ESPÍRITO SANTO LTDA,13.700.575/0001-69
3,Comando do 2º Grupamento de Engenharia,GREEN4T SOLUÇÕES TI LTDA,05.854.663/0001-97
4,07,TARGET ENGENHARIA E CONSULTORIA LTDA,22.409.542/0001-55


In [5]:
import pickle
pickle.dump(contratos, open("contratos_{}.pickle".format(data), "wb"))